In [1]:
from model.TKAN import Model

In [2]:
a = Model()

In [3]:
a.compile_execute_test()

Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 16s 61ms/step - loss: 0.4630
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 0.1630
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step - loss: 0.1096
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 129ms/step - loss: 0.1086
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 127ms/step - loss: 0.0981
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 128ms/step - loss: 0.0888
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 125ms/step - loss: 0.0941
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 125ms/step - loss: 0.0878
Epoch 9/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 123ms/step - loss: 0.0900
Epoch 10/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 128ms/step - loss: 0.0888


In [5]:
import DataExtraction.meteocat as meteocat
meteocat.print_api_usage()

{'client': {'nom': '488 Mireia Miró Fatás UPC'}, 'plans': [{'nom': 'Referència Bàsic', 'periode': 'Mensual', 'maxConsultes': 2000, 'consultesRestants': 2000, 'consultesRealitzades': 0}, {'nom': 'Quota', 'periode': 'Mensual', 'maxConsultes': 300, 'consultesRestants': 298, 'consultesRealitzades': 2}, {'nom': 'XDDE_250', 'periode': 'Mensual', 'maxConsultes': 250, 'consultesRestants': 250, 'consultesRealitzades': 0}, {'nom': 'XEMA_750 OD', 'periode': 'Mensual', 'maxConsultes': 750, 'consultesRestants': 742, 'consultesRealitzades': 8}, {'nom': 'Predicció_100', 'periode': 'Mensual', 'maxConsultes': 100, 'consultesRestants': 100, 'consultesRealitzades': 0}]}
